
![image](https://user-images.githubusercontent.com/45148200/50420336-167c1e80-0836-11e9-8f3c-224f4cab73bd.png)


#  Rossman Stores Sales forecasting

In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from pandas import datetime
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import explained_variance_score
import xgboost
from sklearn.svm import SVR
%matplotlib inline

## Extracting the data

In [2]:
#Lecture des données
train_data = pd.read_csv('dataset/train.csv', sep=',',low_memory=False)
store_data = pd.read_csv('dataset/store.csv', sep=',',low_memory=False)
test_data = pd.read_csv('dataset/test.csv', sep=',',low_memory=False)


## Feature Engineering

The aim here is to grasp every relevant information from the variables given. 

Furthemore, we also decide the way we are forcasting. In other words we can just feed our model the predictors that are the most relevant according to us, but we can also make it a little more elaborate. For example here we have chosen to predict the sales of every stores separatly then gather everything.

First let's focus on the features.

- The Date variable is not really useful but we can get other information such as the Year and the Month.
- We can map variables such as StoreType, Assortment or StateHoliday in order to replce the "str" with integers and thus make it usable
- In order to clean the dataset we can either drop some rows or fill them with what makes sense. Thus we keep only the rows where the stores is open and makes sales


### Features 

- Filling the blanks & NaNs
- Creating new relevant variables
- Mapping The variables

In [3]:
#Filling the blanks or NaNs
store_data.fillna(0, inplace = True)
store_data['CompetitionDistance'].fillna(store_data['CompetitionDistance'].median(), inplace = True)
#Creating new variables

#From the date
train_data['Date'] = pd.to_datetime(train_data['Date'], errors='coerce')

train_data['Year'] = train_data.Date.dt.year
train_data['Month'] = train_data.Date.dt.month
train_data['Day'] = train_data.Date.dt.day
train_data['WeekOfYear'] = train_data.Date.dt.weekofyear

test_data['Date'] = pd.to_datetime(test_data['Date'], errors='coerce')

test_data['Year'] = test_data.Date.dt.year
test_data['Month'] = test_data.Date.dt.month
test_data['Day'] = test_data.Date.dt.day
test_data['WeekOfYear'] = test_data.Date.dt.weekofyear

# Indicate whether the month is in promo interval
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
store_data['month_str'] = train_data.Month.map(month2str)

def check(row):
    if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
        return 1
    else:
        return 0
        
store_data['IsPromoMonth'] =  store_data.apply(lambda row: check(row),axis=1)  

#From the competition 
store_data['CompetitionOpen'] = 12 * (train_data.Year - store_data.CompetitionOpenSinceYear) + \
        (train_data.Month - store_data.CompetitionOpenSinceMonth)
store_data['PromoOpen'] = 12 * (train_data.Year - store_data.Promo2SinceYear) + \
        (train_data.WeekOfYear - store_data.Promo2SinceWeek) / 4.0
store_data['PromoOpen'] = store_data.PromoOpen.apply(lambda x: x if x > 0 else 0)
store_data.loc[store_data.Promo2SinceYear == 0, 'PromoOpen'] = 0



#Mapping the variables
train_data.StateHoliday = train_data.StateHoliday.replace(0,'0')
train_data["HolidayBin"] = train_data.StateHoliday.map({"0": 0, "a": 1, "b": 2, "c": 3})

store_data["StoreTypeBin"] = store_data.StoreType.map({"a": 1, "b": 2, "c": 3, "d": 4})
store_data['AssortmentBin'] = store_data.Assortment.map({"a": 1, "b": 2, "c": 3, "d": 4})

test_data.StateHoliday =test_data.StateHoliday.replace(0,'0')
test_data["HolidayBin"] = test_data.StateHoliday.map({"0": 0, "a": 1, "b": 2, "c": 3})



In [4]:
store_data

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,month_str,IsPromoMonth,CompetitionOpen,PromoOpen,StoreTypeBin,AssortmentBin
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,Jul,0,82.0,0.00,3,1
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",Jul,1,92.0,64.50,1,1
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",Jul,1,103.0,52.25,1,1
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0,Jul,0,70.0,0.00,3,3
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0,Jul,0,3.0,0.00,1,1
5,6,a,a,310.0,12.0,2013.0,0,0.0,0.0,0,Jul,0,19.0,0.00,1,1
6,7,a,c,24000.0,4.0,2013.0,0,0.0,0.0,0,Jul,0,27.0,0.00,1,3
7,8,a,a,7520.0,10.0,2014.0,0,0.0,0.0,0,Jul,0,9.0,0.00,1,1
8,9,a,c,2030.0,8.0,2000.0,0,0.0,0.0,0,Jul,0,179.0,0.00,1,3
9,10,a,a,3160.0,9.0,2009.0,0,0.0,0.0,0,Jul,0,70.0,0.00,1,1


## Cleaning the dataset

- Delete the variable useless after the feature engineering
- Delete the irrelevant states


In [5]:


del train_data['Date']
del train_data['StateHoliday']

del test_data['Date']
del test_data['StateHoliday']

closed_store_data = test_data["Id"][test_data["Open"] == 0].values
test_data = test_data[test_data["Open"] != 0]
train_data= train_data[(train_data["Sales"]!=0) & (train_data["Open"]!=0)]




## Creating the dataset
As we so in the virtualization notebook the sales depends on variables that are in different tables. Therefore we merge the two tables in order to deal with every variables that influences the sales



In [6]:
join_train = train_data.join(store_data.set_index('Store'),on='Store')
join_test = test_data.join(store_data.set_index('Store'),on='Store')
# Creating the sets that are going to be tun through
train_stores = dict(list(pd.get_dummies(join_train).groupby('Store')))
test_stores = dict(list(pd.get_dummies(join_test).groupby('Store')))

#One hot-vector
# If we choose not to map we can use the get_dummies method 
#that creates a column with every none integer elements found
# in that column

#join_test = pd.get_dummies(join_test)


# Train & Predict

For every stores in the train store we first drop some obvious features. Then we proceed through the estimation process


In [7]:

result = pd.Series()
for i in test_stores:  
    store = train_stores[i]
    X_train = store.drop(["Sales", "Store", "Customers"],axis=1)
    Y_train = store["Sales"]
    X_test  = test_stores[i].copy()   
    store_ind = X_test["Id"]
    X_test.drop(["Id","Store"], axis=1,inplace=True)
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())
    estimator =xgboost.XGBRegressor(n_estimators=18, learning_rate=0.15, max_depth=15)
    #estimator=RandomForestRegressor(n_estimators=10, max_depth=13, criterion = 'mse')
    #estimator= SVR(gamma='scale', C=1.0, epsilon=0.2)
    estimator.fit(X_train, Y_train)
    Y_pred = estimator.predict(X_test)
    result = result.append(pd.Series(Y_pred, index=store_ind))


In [8]:
result = result.append(pd.Series(0, index=closed_store_data))
result = pd.DataFrame({ "Id": result.index, "Sales": result.values})
result.to_csv('submission1_3.csv', index=False,header=True)
